In [1]:
import tensorflow as tf

In [2]:
import pydicom as dcm
import numpy as np
import os

In [3]:
def preprocess_dcm(file_path):
    ds = dcm.dcmread(file_path)
    ds = ds.pixel_array.astype(np.float32) / np.max(np.abs(ds.pixel_array))
    return ds

In [4]:
# dataset = tf.data.Dataset.from_tensor_slices(batch)

In [5]:
path = '/home/jupyter/rsna-intracranial-hemorrhage-detection/stage_2_train/'
ind = 0
paths = os.listdir(path)
def input():
    global ind, paths
    batch = []
    if ind >= len(paths):
        print("STOPPING NOWW!")
        raise tf.errors.OutOfRangeError()
    for i, fname in enumerate(paths[ind:]):
        if i == 100: 
            break
        pixel_array = preprocess_dcm(path+fname)
        if pixel_array.shape != (512,512):
            continue
        batch.append(tf.convert_to_tensor(pixel_array))
        ind += 1
        
    # batch = np.array(batch)
    return tf.data.Dataset.from_tensor_slices(batch)
    # return tf.compat.v1.train.limit_epochs(
      # tf.convert_to_tensor(batch, dtype=tf.float32), num_epochs=1)

In [6]:
num_clusters = 2
kmeans = tf.compat.v1.estimator.experimental.KMeans(
    num_clusters=num_clusters, use_mini_batch=True)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpb9ffwubo', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [7]:
tf.autograph.set_verbosity(0)
for _ in range(100):
    kmeans.train(input)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.


2021-12-17 20:32:48.988678: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-17 20:32:50.827173: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38444 MB memory:  -> device: 0, name: A100-SXM4-40GB, pci bus id: 0000:00:04.0, compute capability: 8.0


INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


KeyboardInterrupt: 